In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import sys
print ("Python version: " + sys.version)

import pandas as pd
import numpy as np

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Loading dataset labeled "E-commerce Sales Report" 
# Link: 
import time
time_begin = time.time()

df = pd.read_csv("C:\Data Analyst Portfolio\ShopX\E-commerce Sale Report.csv")

print(f'Run time: {round(((time.time()-time_begin)/60), 3)} mins') 


Python version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Run time: 0.012 mins


In [ ]:
# Inspect dataset
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.00,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.00,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.00,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.00,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.00,IN,NaN,False,NaN,NaN


In [3]:
# Display the shape of the data
print(f"Shape of data: {df.shape}")

Shape of data: (128975, 24)


In [4]:
# Check for duplicate rows
total_duplicates = df.duplicated().sum()
print(f"Total duplicated rows: {total_duplicates}")

Total duplicated rows: 0


In [6]:
# Examine data types
data_types = df.dtypes

print(data_types)

index                   int64
Order ID                  str
Date                      str
Status                    str
Fulfilment                str
Sales Channel             str
ship-service-level        str
Style                     str
SKU                       str
Category                  str
Size                      str
ASIN                      str
Courier Status            str
Qty                     int64
currency                  str
Amount                float64
ship-city                 str
ship-state                str
ship-postal-code      float64
ship-country              str
promotion-ids             str
B2B                      bool
fulfilled-by              str
Unnamed: 22            object
dtype: object


In [7]:
# Identifying any null variables percentage
missing_percentage = df.isnull().sum() / len(df) * 100
print(f"Null value %: {missing_percentage[missing_percentage > 0]}%")


Null value %: Courier Status      5.33
currency            6.04
Amount              6.04
ship-city           0.03
ship-state          0.03
ship-postal-code    0.03
ship-country        0.03
promotion-ids      38.11
fulfilled-by       69.55
Unnamed: 22        38.03
dtype: float64%


In [ ]:
df = df.copy()

# Drop useless columns

df = df.drop(columns=["index", "Unnamed: 22", "fulfilled-by", "ship-country", "currency"])

# Fill NA values
df['Courier Status'].fillna('Cancelled', inplace=True)
df['Amount'].fillna(0, inplace=True)
df['ship-city'].fillna('Unknown-city', inplace=True)
df['ship-state'].fillna('Unknown-state', inplace=True)
df['ship-postal-code'].fillna(0, inplace=True)
df['promotion-ids'].fillna('No promotion', inplace=True)


0                                              No promotion
1         Amazon PLCC Free-Financing Universal Merchant ...
2              IN Core Free Shipping 2015/04/08 23-48-5-108
3                                              No promotion
4                                              No promotion
                                ...                        
128970                                         No promotion
128971         IN Core Free Shipping 2015/04/08 23-48-5-108
128972                                         No promotion
128973         IN Core Free Shipping 2015/04/08 23-48-5-108
128974         IN Core Free Shipping 2015/04/08 23-48-5-108
Name: promotion-ids, Length: 128975, dtype: str

In [ ]:
# Change the data type for the columns
df['Amount'] = df['Amount'].astype('Float64')

In [20]:
from datetime import datetime

# Change columns from String to Date datatype.
# Format the data if it is either 01-03-22 or 01/03/22
def parse_mixed(s):
    for fmt in ("%d/%m/%y", "%m-%d-%y"):
        try:
            return pd.to_datetime(s, format=fmt)
        except ValueError:
            pass
    return pd.NaT  # or raise

df["Date"] = df["Date"].apply(parse_mixed)

In [21]:
# Strips any leading/trailing spaces in any column with string datatype

df = df.apply(lambda s: s.str.strip() if s.dtype == "object" else s)

In [22]:
df.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,647.62,MUMBAI,MAHARASHTRA,400081.00,NaN,False
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.00,BENGALURU,KARNATAKA,560085.00,Amazon PLCC Free-Financing Universal Merchant ...,False
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,753.33,PUDUCHERRY,PUDUCHERRY,605008.00,NaN,False
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.00,CHENNAI,TAMIL NADU,600073.00,NaN,False


In [26]:
# Standardize text columns

text_cols = [
    "Status", "Fulfilment", "ship-service-level",
    "Style", "SKU", "Category", "Size",
    "ship-city", "ship-state", "Courier Status"
]

# For all the values in these string columns, strip any leading/trailing spaces
# replace any unnecessary spaces in between words
# lastly set all values as UPPER case
for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.upper()
    )

In [27]:
# Create aggregate columns
df["gross_revenue"] = df["Qty"] * df["Amount"]
df["is_cancelled"] = df["Status"].str.contains("CANCELLED")
df["is_delivered"] = df["Status"].str.contains("DELIVERED")
df["order_month"] = df["Date"].dt.to_period('M')


In [33]:
df.to_csv("E-commerce_Sale_Report_cleaned.csv", index=False)